In [1]:
!pip install -q noisereduce

In [2]:
import os
import gc
import cv2
import glob
import torch
import shutil
import random
import torchaudio
import numpy as np
import pandas as pd
import noisereduce as nr
import matplotlib.pyplot as plt
from math import ceil
from pprint import pprint
from tqdm.auto import tqdm
from functools import partial
from joblib import Parallel, delayed
from torch.utils.data import DataLoader

In [3]:
PATH_DATASET = "/kaggle/input/birdclef-2022/"
train_meta   = pd.read_csv(PATH_DATASET + "train_metadata.csv")
train_meta.head(3)

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,time,url,filename
0,afrsil1,[],"['call', 'flight call']",12.3910,-1.4930,Euodice cantans,African Silverbill,Bram Piot,Creative Commons Attribution-NonCommercial-Sha...,2.5,08:00,https://www.xeno-canto.org/125458,afrsil1/XC125458.ogg
1,afrsil1,"['houspa', 'redava', 'zebdov']",['call'],19.8801,-155.7254,Euodice cantans,African Silverbill,Dan Lane,Creative Commons Attribution-NonCommercial-Sha...,3.5,08:30,https://www.xeno-canto.org/175522,afrsil1/XC175522.ogg
2,afrsil1,[],"['call', 'song']",16.2901,-16.0321,Euodice cantans,African Silverbill,Bram Piot,Creative Commons Attribution-NonCommercial-Sha...,4.0,11:30,https://www.xeno-canto.org/177993,afrsil1/XC177993.ogg


In [4]:
def get_length(fn):
    fp = os.path.join(PATH_DATASET, "train_audio", fn)
    waveform, sample_rate = torchaudio.load(fp)
    return waveform.size()[-1]

sizes = Parallel(n_jobs=os.cpu_count())(delayed(get_length)(fn) for fn in tqdm(train_meta["filename"]))

  0%|          | 0/14852 [00:00<?, ?it/s]

In [5]:
df_order = pd.DataFrame({"path"  : train_meta["filename"],
                         "rating": train_meta["rating"],
                         "size"  : sizes})
print(df_order["size"].max())
df_order = df_order[(df_order.rating >= 2)&(df_order["size"] < 63262302)]
print(df_order["size"].max())
paths = df_order.sort_values("size").path[::-1]
len(paths)

143569152
56241408


13940

# Convert audio to set of spectrograms

In [6]:
@torch.no_grad()
def create_spectrogram(fname, reduce_noise=False, frame_length=5, channel=0, device="cpu"):
    # Define transform for spectrogram
    transform = torchaudio.transforms.Spectrogram(n_fft=1800, win_length=512).to(device)
    # Load audio data
    waveform, sample_rate = torchaudio.load(fname)
    if reduce_noise:
        waveform = torch.tensor(nr.reduce_noise(
            y=waveform,
            sr=sample_rate,
            win_length=transform.win_length,
            use_tqdm=False,
            n_jobs=2,
        ))
    # Return if the audio is too short
    nb = int(frame_length * sample_rate)
    if nb*0.5 > waveform.size()[-1]:
        print(nb, waveform.size()[-1])
        return []
    # Trans to spectrogram
    spectrograms = []
    print(ceil(waveform.size()[-1] / nb))
    for i in range(ceil(waveform.size()[-1] / nb)):
        frame = waveform[channel][i*nb:(i+1)*nb]
        if len(frame) < nb:
            if i == 0:
                rep   = round(float(nb) / len(frame))
                frame = frame.repeat(int(rep))
            elif len(frame) < (nb * 0.33):
                continue
            else:
                frame = waveform[channel][-nb:]
        sg = torch.log(transform(frame.to(device))).cpu()
        spectrograms.append(np.nan_to_num(sg.numpy()))
    return spectrograms

In [7]:
img_extension = ".jpg"

def convert_and_export(fn, path_in, path_out, reduce_noise=False, frame_length=5, device="cpu"):
    path_audio = os.path.join(path_in, fn)
    try:
        sgs = create_spectrogram(
            path_audio,
            reduce_noise=reduce_noise,
            frame_length=frame_length,
            device=device
        )
    except Exception as ex:
        print(f"Failed conversion for audio: {path_audio}")
        return
    if not sgs:
        print(f"Too short audio for: {path_audio}")
        return
    
    # see: https://www.kaggle.com/c/birdclef-2022/discussion/308861
    if len(sgs) > 2:
        sgs = [sgs[0], sgs[-1]]
    for i, sg in enumerate(sgs):
        path_img = os.path.join(path_out, fn + f".{i:03}" + img_extension)
        os.makedirs(os.path.dirname(path_img), exist_ok=True)
        plt.imsave(path_img, sg, vmin=-70, vmax=20)

# Run conversion

In [8]:
_convert_and_export = partial(
    convert_and_export,
    path_in=os.path.join(PATH_DATASET, "train_audio"),
    path_out="train_images",
    reduce_noise=True,
    frame_length=5,
    device="cuda" if torch.cuda.is_available() else "cpu"
)

_ = Parallel(n_jobs=3)(delayed(_convert_and_export)(fn) for fn in tqdm(paths))

  0%|          | 0/13940 [00:00<?, ?it/s]

 32%|███▏      | 30/94 [00:37<01:19,  1.25s/it]

218



/opt/conda/lib/python3.7/site-packages/matplotlib/colors.py:621: RuntimeWarning: overflow encountered in multiply
  xa *= self.N
 63%|██████▎   | 34/54 [00:43<00:24,  1.25s/it]/opt/conda/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
 33%|███▎      | 16/49 [00:20<00:42,  1.28s/it]

200


...

4



100%|██████████| 2/2 [00:00<00:00, 548.49it/s]

4
4



100%|██████████| 2/2 [00:00<00:00, 51.56it/s]

4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


In [9]:
print(len(glob.glob("./train_images/*/*.jpg")))

25880


In [10]:
shutil.make_archive('./train_images', 'zip', root_dir='./train_images')

'/kaggle/working/train_images.zip'

In [11]:
!rm -rf ./train_images/